In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
t = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")
t.head().T

In [ ]:
enceinte = (t.Pregnancies > 0)
t[enceinte].head().T

In [ ]:
t[enceinte].Outcome.value_counts()

In [ ]:
t.Outcome.value_counts()

In [ ]:
t.count()

On remarque donc que, dans cette table, il n'y a pas de valeurs manquantes.
Les tests ci-dessus correspondent simplement à des tests pour compter le nombre de diabétiques et non-diabétiques en général et parmi celles qui ont été enceintes au moins une fois.

On va désormais essayer de créer un algorithme d'apprentissage et de test des données.

In [ ]:
X = t.drop(['Outcome'], axis=1)
y = t.Outcome

On sépare le dataset en deux parties :

un ensemble d'apprentissage (entre 70% et 90% des données), qui va permettre d'entraîner le modèle
un ensemble de test (entre 10% et 30% des données), qui va permettre d'estimer la pertinence de la prédiction

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

Donc, sur nos 768 individus, on va en utiliser 614 pour créer et entrainer notre algorithme, et 154 pour le tester. On va désormais s'occuper de la régression logistique.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

In [ ]:
y_lr = lr.predict(X_test)

Ici, il y a un avertissement avec la régression logistique, elle ne semble pas se faire correctement.
On va désormais mesurer les performances.

In [ ]:
# Importation des méthodes de mesure de performances
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

In [ ]:
print(confusion_matrix(y_test,y_lr))

On a donc avec cet algorithme: 89 Vrais Positifs et 31 Vrais Négatifs, pour 24 Faux Positifs et 10 Faux Négatifs.

On a donc une précision qui semble assez bonne.

Ici, les termes "positif" et "négatif" n'ont rien à voir avec le médical, mais seulement avec la mesure de performances. Ici, on a cherché les personnes non diabétiques (Outcome = 0); par exemple, l'algorithme a prédit que 113 des 154 individus testés ne seraient pas diabétiques. 89 ne présentaient en effet pas de diabète, et 24 étaient diabétiques.

In [ ]:
print(accuracy_score(y_test,y_lr))

Néanmoins cette mesure peut être faussée dans certains cas, en particulier si le nombre de 0 et de 1 est déséquilibré, **ce qui est le cas dans notre exemple**. On a donc d'autres estimateurs :

la précision est le nombre de prédictions positives correctes sur le nombre total de prédictions positives : precision = VP/(VP+FP)
la sensibilité (recall) est le nombre de prédictions positives sur le nombre effectif de "oui" : recall = VP:(VP+FN)
le score F1 est la moyenne pondérée de la précision et de la sensibilité : f1-score = 2xprecisionxrecall/(precision+recall)

In [ ]:
print(classification_report(y_test, y_lr))

Ci-dessous, on va imprimer les probabilités de diabète ou non (Outcome = 0 ou 1) pour chaque individu Test.

In [ ]:
probas_indiv = lr.predict_proba(X_test)
print (probas_indiv)

On met les probabilités de prédiction de la valeur 1 dans un dataframe, avec les valeurs effectives, pour faciliter la visualisation :

In [ ]:
dfprobas = pd.DataFrame(probas_indiv,columns=['proba_0','proba_1'])
dfprobas['y'] = np.array(y_test)

In [ ]:
dfprobas

In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(1-dfprobas.proba_0[dfprobas.y==0], bins=50)
sns.distplot(dfprobas.proba_1[dfprobas.y==1], bins=50)

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,probas_indiv[:, 1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print (roc_auc)

In [ ]:
plt.figure(figsize=(12,12))
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')        # plus mauvaise courbe
plt.plot([0,0,1],[0,1,1],'g:')     # meilleure courbe
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

Ajustement des hyper paramètres

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf))

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

In [ ]:
rf1 = ensemble.RandomForestClassifier(n_estimators=10, min_samples_leaf=10, max_features=3)
rf1.fit(X_train, y_train)
y_rf1 = rf.predict(X_test)
print(classification_report(y_test, y_rf1))

In [ ]:
from sklearn.model_selection import validation_curve
params = np.arange(1, 300,step=30)
train_score, val_score = validation_curve(rf, X, y, 'n_estimators', params, cv=7)
plt.figure(figsize=(12,12))
plt.plot(params, np.median(train_score, 1), color='blue', label='training score')
plt.plot(params, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('n_estimators')
plt.ylabel('score');

La méthode GridSearchCV permet de tester plusieurs combinaisons de paramètres (listés dans une grille de paramètres) et de sélectionner celle qui donne la meilleure pertinence

In [ ]:
from sklearn import model_selection
param_grid = {
              'n_estimators': [10, 100, 500],
              'min_samples_leaf': [1, 20, 50]
             }
estimator = ensemble.RandomForestClassifier()
rf_gs = model_selection.GridSearchCV(estimator, param_grid)

Ici on a choisi des valeurs pour le nombres d'arbres dans la forêt aléatoire ('n_estimators') et le nombre minimum d'échantillons pour une feuille.

In [ ]:
rf_gs.fit(X_train, y_train)

On peut voir les paramètres sélectionnés et le score :

In [ ]:
print(rf_gs.best_params_)

On sélectionne le meilleur estimateur :

In [ ]:
rf2 = rf_gs.best_estimator_
y_rf2 = rf2.predict(X_test)
print(classification_report(y_test, y_rf2))

Notre modèle est un peu plus performant désormais.

IMPORTANCE DES CARACTERISTIQUES

In [ ]:
importances = rf2.feature_importances_
indices = np.argsort(importances)

In [ ]:
plt.figure(figsize=(8,5))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), X_train.columns[indices])
plt.title('Importance des caracteristiques')

On voit donc que (sans surprise), le paramètre le plus corrélé au diabète est l'absorption de glucose par l'individu.

Suivent ensuite le BMI et l'age, qui sont les 2 autres paramètres les plus significatifs.

Désormais nous allons tester la méthode XG Boost.

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBClassifier()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
cm = confusion_matrix(y_test, y_xgb)
print(cm)
print(classification_report(y_test, y_xgb))

On remarque ici que notre précision est un peu moins grande avec cette méthode, ce qui est rare puisque la méthode XG Boost est généralement la plus efficace.